# 检测画面中的运动物体

此文档用来分析画面中是否有运动或出现变化的物体，基于openCV。


# 导入摄像头相关的函数库

运行以下代码块后，稍等片刻，等待摄像头初始化，初始化成功后会在该代码块的下方出现一个300x300大小的摄像头实时视频画面。

你可以在这个画面上点击右键，点击`Create New View for Output`，这样就可以将摄像头画面放置在窗口的一遍，即使浏览到该文档的后面部分，你依然可以随时观看摄像头的画面，该方法也同样适用于其它组件。

多次运行该代码块有一定概率会初始化失败，在`jetbot.Camera`中已经包含了该问题的解决方法，你只需要重新运行该Kernel即可，但是注意不要使用该选项卡上方的圆形箭头来重新运行，那种方法有一定概率会依然初始化失败。

建议重新运行该Kernel的方法：
在左侧`File Browser`中，前方有绿色圆点的`*.ipynb`文件上点击右键（前面有绿色的原点说明Kernel正在运行中），选择`Shut Down Kernel`，你会发现绿色的圆点消失了，然后关闭掉该选项卡再双击刚才被关掉的`*.ipynb`文件来重新运行kernel。

再次运行以下代码块，摄像头就应该可以正常初始化了。

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# 运动侦测的函数

该运动侦测功能基于openCV来实现，在Jetpack中已经预安装了openCV以及imutils，所以你可以直接运行以下代码块导入所需要的函数库，如果你使用的不是jetpack或者报错没有相应的库，则需要手动在终端中安装openCV或imutils，分别使用`sudo pip3 install opencv-python`和`sudo pip3 install imutils`来安装相应的库，如果运行没有报错提示缺少这两个库，则可以忽略掉这些内容直接进行下一步的代码块运行。

In [2]:
import cv2
import imutils
import datetime

# avg用来保存一帧基准画面（背景），新的画面与其比较判断画面中的哪里发生了变化。
avg = None


lastMovtionCaptured = datetime.datetime.now()

# 在这个函数中导入采集到的画面帧，加以处理。
def motionDetect(imgInput):
    global avg, lastMovtionCaptured
    
    # 获得当前时间戳。
    timestamp = datetime.datetime.now()
    
    # 将画面转换为黑白，可以增大分析效率。
    gray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    
    # 将画面增加高斯模糊，避免噪点带来的误判。
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # 如果还没有获得基准帧（背景），则新建一个。
    if avg is None:
        avg = gray.copy().astype("float")
        return imgInput

    # 背景更新。
    cv2.accumulateWeighted(gray, avg, 0.5)
    
    # 比较新画面gray与背景的不同。
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    # 获得画面中出现变化的区域轮廓。
    thresh = cv2.threshold(frameDelta, 5, 255,
        cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # 画面中也许不止有一个出现变化的区域，所以需要使用for循环来圈出所有轮廓。
    for c in cnts:
        # 这里默认是30，是变化面积的阈值，我们只分析大于800的区域。
        # 此数值越小，运动检测越灵敏，但也可能会检测到无意义的噪音。
        if cv2.contourArea(c) < 30:
            continue

        # 绘制元素，包括轮廓线和文字
        (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
        cv2.rectangle(imgInput, (mov_x, mov_y), (mov_x+mov_w, mov_y+mov_h), (128, 255, 0), 1)

        # 保存当前的时间戳用于标记变化检测到的时间。
        lastMovtionCaptured = timestamp

    # 为了避免绘制元素闪烁频率过高，将检测到运动后的0.5秒内也算作检测到了运动，并绘制元素。
    if (timestamp - lastMovtionCaptured).seconds >= 0.5:
        cv2.putText(imgInput,"Motion Detecting",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(128,255,0),1,cv2.LINE_AA)
    else:
        cv2.putText(imgInput,"Motion Detected",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,128,255),1,cv2.LINE_AA)
    
    # 返回处理后的画面
    return imgInput

# 处理视频帧并显示

运行以下代码块后，你就可以看到画面的颜色发生了变化，说明视频画面已经通过`motionDetect()`函数处理成功。

In [3]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(motionDetect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

此时你已经运行了全部代码块，当画面中有物体运动或发生变化时，文字内容会发生改变，会有绿色的矩形框圈出发生变化的区域。

# 关闭图像处理和摄像头
运行以下代码块来关闭该图像处理功能。

In [4]:
camera.unobserve(execute, names='value')

在结束该例程前，我们需要关闭摄像头，这样才可以释放摄像头资源供其它例程使用。

In [4]:
camera.stop()